In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier



In [2]:
df = pd.read_csv("train.csv")

In [3]:
df = df.drop(columns=['ID'])
df = df.drop(columns=['Payment Plan'])
df = df.drop(columns=['Accounts Delinquent'])
df = df.drop(columns=['Batch Enrolled'])
df =df.drop(columns=['Application Type'])

In [4]:
df['Loan Title'] = df['Loan Title'].str.lower()
len(df['Loan Title'].unique())
df['Loan Title'].unique()

array(['debt consolidation', 'credit card refinancing',
       'home improvement', 'credit consolidation', 'green loan', 'other',
       'moving and relocation', 'credit cards', 'medical expenses',
       'refinance', 'credit card consolidation', 'lending club',
       'debt consolidation loan', 'major purchase', 'vacation',
       'business', 'credit card payoff', 'credit card',
       'credit card refi', 'personal loan', 'cc refi', 'consolidate',
       'medical', 'loan 1', 'consolidation', 'card consolidation',
       'car financing', 'debt', 'home buying', 'freedom', 'consolidated',
       'get out of debt', 'consolidation loan', 'dept consolidation',
       'personal', 'cards', 'bathroom', 'refi', 'credit card loan',
       'credit card debt', 'house', 'debt consolidation 2013',
       'debt loan', 'cc refinance', 'home', 'cc consolidation',
       'credit card refinance', 'credit loan', 'payoff',
       'bill consolidation', 'credit card paydown', 'credit card pay off',
       'g

In [5]:
map_dict = {'Loan Title':{'debt consolidation':'debt consolidation', \
                'debt consolidation loan':'debt consolidation', \
                'debt consolidation 2013':'debt consolidation', \
                'dept consolidation':'debt consolidation' ,
                'credit card refinancing':'credit card refinancing', \
                'refinance':'credit card refinancing', \
                'cc refinance':'credit card refinancing',\
                'credit card refinance':'credit card refinancing',\
                'cc-refinance':'credit card refinancing', \
                'refinance loan':'credit card refinancing', \
                'credit card refinance loan':'credit card refinancing',\
                'credit card refi':'credit card refinancing', \
                'cc refi':'credit card refinancing', \
                'refi':'credit card refinancing'  ,
                'home improvement':'home improvement',\
                'home improvement loan':'home improvement',\
                'bathroom':'home improvement',\
                'pool':'home improvement' ,
                'home buying':'home buy',\
                'house':'home buy',\
                'home':'home buy',\
                'home loan':'home buy' ,
                'credit consolidation':'credit consolidation',\
                'consolidation loan':'credit consolidation',\
                'cc consolidation':'credit consolidation',\
                'conso':'credit consolidation',\
                'credit card consolidation':'credit consolidation',\
                'card consolidation':'credit consolidation',\
                'loan consolidation':'credit consolidation',\
                'consolidate':'credit consolidation',\
                'consolidated':'credit consolidation' ,
                'medical':'medical purpose',\
                'medical expenses':'medical purpose',\
                'medical loan':'medical purpose'  ,
                'lending club' : 'lending purpose' , \
                'lending loan' : 'lending purpose'  ,
                'business' : 'business purpose' , \
                'green loan' : 'business purpose' , \
                'moving and relocation' : 'business purpose'  , \
                'credit cards' : 'credit cards payoff' , \
                'credit card' : 'credit cards payoff' , \
                'credit card loan' : 'credit cards payoff' , \
                'credit' : 'credit cards payoff' , \
                'credit card debt' : 'credit cards payoff' , \
                'cc loan' : 'credit cards payoff' , \
                'cc' : 'credit cards payoff' , \
                'cards' : 'credit cards payoff' , \
                'credit card payoff' : 'credit cards payoff' , \
                'credit pay off' : 'credit cards payoff' , \
                'credit payoff' : 'credit cards payoff' , \
                'credit loan' : 'credit cards payoff' , \
                'payoff' : 'credit cards payoff' , \
                'pay off' : 'credit cards payoff' , \
                'credit card paydown' : 'credit cards payoff' , \
                'credit card pay off' : 'credit cards payoff' ,
                'personal loan' : 'perosonal purpose' , \
                'personal' : 'perosonal purpose' , \
                'wedding loan' : 'perosonal purpose' , \
                'getting ahead' : 'perosonal purpose' , \
                'vacation' : 'perosonal purpose' , \
                'major purchase' : 'perosonal purpose'  , \
                'car financing' : 'vehicle purpose' , \
                'car loan' : 'vehicle purpose' , \
                'debt' : 'debt payoff' , \
                'get out of debt' : 'debt payoff' , \
                'debt loan' : 'debt payoff' , \
                'get debt free' : 'debt payoff' , \
                'debt payoff' : 'debt payoff' , \
                'debt free' : 'debt payoff' , \
                'freedom' : 'debt payoff' , \
                'relief' : 'debt payoff' , \
                'debt reduction' : 'debt payoff' , \
                'bill consolidation' : 'bill payoff' , \
                'bill payoff' : 'bill payoff' , \
                'pay off bills' : 'bill payoff' , \
                'bills' : 'bill payoff'  , \
                'other' : 'other purpose' , \
                'loan 1' : 'other purpose' , \
                'loan1' : 'other purpose' , \
                'loan' : 'other purpose' , \
                'myloan' : 'other purpose' , \
                'my loan' : 'other purpose' }
            }

In [6]:
df.replace(map_dict, inplace=True)
len(df['Loan Title'].unique())


15

In [7]:

colunas = ['Home Ownership', 'Interest Rate', 'Inquires - six months', 'Total Accounts',
               'Total Received Interest', 'Total Received Late Fee', 'Recoveries', 
               'Collection Recovery Fee', 'Collection 12 months Medical', 
               'Total Collection Amount', 'Total Revolving Credit Limit']

#Loop em relação as features
for coluna in colunas:
  Q1 = df[coluna].quantile(q = 0.25) #Definindo o primeiro quartil
  Q3 = df[coluna].quantile(q = 0.75) #Definindo o segundo quartil
  IQR = Q3 - Q1 #Definindo o interquartil
  superior = Q3 + IQR  #Definindo o Limite Superior
  inferior = Q1 - IQR #Definindo o Limite Inferior

  #Eliminando os outliers acima do limite superior
  df.drop( df.loc[ df[coluna] > superior ].index, axis = 0, inplace = True )

  #Eliminando os outliers abaixo do limite inferior
  df.drop( df.loc[ df[coluna] < inferior ].index, axis = 0, inplace = True )

In [8]:
df.drop(['Collection 12 months Medical', 'Inquires - six months'], axis = 1, inplace = True)
print(df.head())

    Loan Amount  Funded Amount  Funded Amount Investor  Term  Interest Rate  \
1          3609          11940            12191.996920    59      12.237563   
7         20744          10609             7645.014802    58      13.993688   
8          9299          11238            13429.456610    59      11.178457   
9         19232           8962             7004.097481    58       5.520413   
11        16581           8767            10637.049030    59       9.535988   

   Grade Sub Grade Employment Duration  Home Ownership Verification Status  \
1      C        D3                RENT     39833.92100     Source Verified   
7      A        A5                 OWN     61723.52014        Not Verified   
8      G        C2            MORTGAGE     63205.09072            Verified   
9      C        B5                RENT     42015.46586     Source Verified   
11     A        D4            MORTGAGE     39605.50605     Source Verified   

    ... Initial List Status  Total Received Interest  To

In [9]:
import pandas as pd
df=pd.read_csv("train_with_dummies_0_1.csv")
X = df.drop(['Loan Status'], axis = 1)
y = df['Loan Status']

In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.neighbors import KNeighborsClassifier

print(df.head())

#Instanciando o objeto RandomUnderSampler()
rus = RandomUnderSampler()

# #Balanceando a amostra
X_train, y_train = rus.fit_resample(X, y)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, 
                                                    y_train, 
                                                    shuffle = True, 
                                                    stratify = y_train,
                                                    random_state = 42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

knn_model = KNeighborsClassifier(n_neighbors=10)  # Escolha o número desejado de vizinhos

knn_model.fit(X_train, y_train)

y_pred = knn_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Exibindo os resultados
print(f"Acurácia: {accuracy}")
print("\nMatriz de Confusão:")
print(conf_matrix)
print("\nRelatório de Classificação:")
print(class_report)


   Loan Amount  Funded Amount  Funded Amount Investor  Term  Interest Rate  \
0        10000          32236             12329.36286    59      11.135007   
1         3609          11940             12191.99692    59      12.237563   
2        28276           9311             21603.22455    59      12.545884   
3        11170           6954             17877.15585    59      16.731201   
4        16890          13226             13539.92667    59      15.008300   

   Home Ownership.1  Debit to Income  Delinquency - two years  \
0      176346.62670        16.284758                        1   
1       39833.92100        15.412409                        0   
2       91506.69105        28.137619                        0   
3      108286.57590        18.043730                        1   
4       44234.82545        17.209886                        1   

   Inquires - six months  Open Account  ...  Loan Title_personalloan  \
0                      0            13  ...                        0

In [11]:

# Criando o modelo SVM
svm_model = SVC()  # Você pode ajustar o kernel conforme necessário

# Treinando o modelo SVM
svm_model.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred_svm = svm_model.predict(X_test)

# Avaliando o desempenho do modelo SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
class_report_svm = classification_report(y_test, y_pred_svm)

# Exibindo os resultados do modelo SVM
print("Resultados do Modelo SVM:")
print(f"Acurácia: {accuracy_svm}")
print("\nMatriz de Confusão:")
print(conf_matrix_svm)
print("\nRelatório de Classificação:")
print(class_report_svm)


Resultados do Modelo SVM:
Acurácia: 0.5070084100921105

Matriz de Confusão:
[[727 550]
 [681 539]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.52      0.57      0.54      1277
           1       0.49      0.44      0.47      1220

    accuracy                           0.51      2497
   macro avg       0.51      0.51      0.50      2497
weighted avg       0.51      0.51      0.51      2497

